In [ ]:
# Chios
import numpy as np 
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,RepeatedKFold

In [ ]:
train = pd.read_csv('/content/houseprice/train_processed.csv') 
test = pd.read_csv('/content/houseprice/test_processed.csv') 

In [ ]:
target_variable = train.resale_price.values
train = train.drop(['resale_price'], axis=1)

In [ ]:
kf = KFold(n_splits=10, random_state=42, shuffle=True)
cv_scores = []
cv_std = []
    
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model):
    rmse = np.sqrt(-cross_val_score(model, train, target_variable, scoring="neg_mean_squared_error", cv=kf))
    return (rmse)

In [ ]:
gb = GradientBoostingRegressor(
    max_depth=10,
    learning_rate=0.05,
    max_features=0.8,
    random_state=42,
    loss='huber',
    subsample=0.7
)

In [ ]:
# This is what you need to run
rf_score = cv_rmse(rf)
print("mean: {}".format(rf_score.mean()))
print("std: {}".format(rf_score.std()))

In [ ]:
gb.fit(train, target_variable)

In [ ]:

data = pd.DataFrame({
    'prediction': np.expm1(gb.predict(train)).reshape(1, -1)[0],
    'target': np.expm1(target_variable)
})
sns.regplot(x='prediction', y='target', palette='Blues', data=data)

In [ ]:
feature_scores = pd.Series(gb.feature_importances_, index=train.columns).sort_values(ascending=False)
f, ax = plt.subplots(figsize=(30, 24))
ax = sns.barplot(x=feature_scores, y=feature_scores.index)
ax.set_title("Visualize feature scores of the features")
ax.set_yticklabels(feature_scores.index)
ax.set_xlabel("Feature importance score")
ax.set_ylabel("Features")
plt.show()
plt.savefig("GBR-Features Importances")

In [ ]:
prediction = np.expm1(gb.predict(test))